In [1]:
import pandas as pd


In [2]:
import numpy as np

In [6]:

point_elevation = pd.read_csv('data/raw_data/elevation_segid_dc.csv')
point_elevation.columns = ['lat', 'lon', 'ele', 'STREETSEGID']

In [24]:
import geopandas as gpd

In [25]:
segs = gpd.read_file('data/segments_dc.geojson')

In [34]:
def min_max_ele(df):
    max_ = df.ele.max()
    min_ = df.ele.min()
    return max_ - min_


In [35]:
diff = point_elevation.groupby('STREETSEGID').apply(min_max_ele)

In [45]:
ftr = segs[['STREETSEGID']].merge(diff.to_frame().reset_index()).drop('STREETSEGID', axis=1)

In [46]:
ftr.index.name = 'index_seg'
ftr.columns = ['elevation_diff']

In [48]:
ftr.to_csv('data/feature_elevation_diff_dc.csv')

In [8]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    km = 6367 * c
    m = km * 1000
    return m

def dist_height(pt1, pt2):
    distance = haversine(pt1['lon'], pt1['lat'], pt2['lon'], pt2['lat'])
    height = pt2['ele'] - pt1['ele']
    return distance, height

def get_dist_heights(df):
    distances = []
    heights = []
    for i in range(df.shape[0]-1):
        pt1 = df.iloc[i]
        pt2 = df.iloc[i+1]
        d, h = dist_height(pt1, pt2)
        distances.append(d)
        heights.append(h)
    return [distances, heights]


def get_sin(row):
    distances = row.distances
    heights = row.heights
    return [heights[i]/distances[i] for i in range(len(distances))]


In [9]:
res = point_elevation.groupby('STREETSEGID').apply(get_dist_heights)

df = res.to_frame()
df['distances'] = df[0].apply(lambda x: x[0])
df['heights'] = df[0].apply(lambda x: x[1])
df['sins'] = df.apply(get_sin, axis=1)

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


In [32]:
segs[segs.STREETSEGID==11712].geometry.values[0].wkt

'LINESTRING (-77.00593551016078 38.91813016303549, -77.00593459207259 38.91818304198257, -77.00593367440291 38.91824096559412, -77.00593633667745 38.91836239775159, -77.00593754883516 38.91851748622692, -77.0059375634096 38.91851941288275, -77.00593756629235 38.91851941288264)'

In [14]:
df['sins'].apply(max).argmax()

11712

In [22]:
df.loc[11712, ['distances', 'heights']].values

array([ [5.876715107985784, 6.437256885091748, 13.496125320024412, 17.234543074791613, 0.2141035554083236, 0.00024924334173302476],
       [0.45000000000000284, 0.58999999999999631, 0.50999999999999801, 0.5, 1.5300000000000011, 0.0]], dtype=object)

In [33]:
point_elevation[point_elevation.STREETSEGID==11712]

,lat,lon,ele,STREETSEGID
102550,38.918130,-77.005936,52.36,11712
102551,38.918183,-77.005935,52.81,11712
102552,38.918241,-77.005934,53.40,11712
102553,38.918362,-77.005936,53.91,11712
102554,38.918517,-77.005938,54.41,11712
102555,38.918519,-77.005938,55.94,11712
102556,38.918519,-77.005938,55.94,11712
